## 5.4 単純なレイヤの実装

### 5.4.1 乗算レイヤの実装

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout): 
        dx = dout * self.y # xとyをひっくり返す
        dy = dout * self.x
        
        return dx, dy

In [2]:
apple = 100
apple_num = 2
tax = 1.1

#layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price) # 220

220.00000000000003


In [3]:
# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax) # 2.2 110 200

2.2 110.00000000000001 200


### 加算レイヤの実装

In [4]:
class AddLayer:
    def __init__(self): 
        pass  # passは何もしないと言う命令
    
    def forward(self, x, y): 
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1 

#layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)               #(1)
orange_price = mul_orange_layer.forward(orange, orange_num)           #(2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price) #(3)
price = mul_tax_layer.forward(all_price, tax)                         #(4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)                         #(4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) #(3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)           #(2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)               #(1)

print(price) #715
print(dapple_num, dapple, dorange, dorange_num, dtax) # 110 2.2 3.3 165 650

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


## 5.5 活性化関数レイヤの実装

### 5.5.1 ReLUレイヤ

In [6]:
class Relu: 
    def __init__(self):
        self.mask = None # mask: インスタンス変数，True, FalseからなるNumPy配列
                         # 順伝播の入力であるxの要素で0以上の場合True, それ以外（0より小さい）をFalse   
    
    def forward(self, x): 
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout): 
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [7]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [8]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


### 5.5.2 Sigmoidレイヤ

In [9]:
class Sigmoid:
    def __init__(self, x): 
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out # 順伝播出力をインスタンス変数のoutに保持して，逆伝播時にoutを使って計算する
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx